# DATA INPUT!

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box,Polygon
import analysis_tool as tool

gdfs = tool.load_all_gdfs("origin_data")
# print(len(gdfs))
# 文件内容为：gdfs["文件夹名"]
# ├── 00_Boundaries/                     # 边界
# │
# ├── 01_production_farming/             # 农作物（可耕地）
# │
# ├── 02_production_aquaculture/         # 水产养殖（海岸泻湖 + 河口）
# │
# ├── 03_production_livestock/           # 畜牧养殖（草场）
# │
# ├── 04_transformation_industry/        # 食品加工厂（GOOGLE 来源）
# │
# ├── 05_transformation_energy/          # 水厂、电厂、供热等能源基础设施
# │
# ├── 06_logistics_transportation/        # 公共交通 + 港口 + 仓储 + 配送中心
# │
# ├── 07_logistics_roads/                # 可通行道路（MultiDiGraph）
# │
# ├── 08_supply_retail/                  # 超市、市场、批发商等零售终端
# │
# ├── 09_supply_public_services/         # 公共服务设施：学校、医疗、福利
# │
# ├── 10_built_environment/              # 建成区信息（居住、商业、工业）
# │
# ├── 11_consumption_facilities/         # 餐饮、酒店、住宿类消费终端
# │
# ├── 12_consumption_landuse_mix/        # 土地混合度计算所需的 landuse 数据
# │
# ├── 13_waste_treatment/                # 废弃物处理设施：垃圾厂、废水厂


✅  已加载 00_Boundaries: Whole area.shp (1 条记录, 0.08 MB)
✅  已加载 01_production_farming: farming.shp (1647 条记录, 2.90 MB)
✅  已加载 02_production_aquaculture: aquaculture.shp (211 条记录, 2.49 MB)
✅  已加载 03_production_livestock: livestock.shp (1538 条记录, 1.61 MB)
✅  已加载 04_transformation_industry: processing.shp (103 条记录, 0.00 MB)
✅  已加载 05_transformation_energy: energy.shp (147 条记录, 0.03 MB)
✅  已加载 06_logistics_transportation: transportation.shp (656 条记录, 0.28 MB)
✅  已加载 07_logistics_roads: roads_highways.shp (58252 条记录, 6.92 MB)
✅  已加载 08_supply_retail: shop_food_beverage_data.shp (432 条记录, 0.01 MB)
✅  已加载 09_supply_public_services: public_services.shp (911 条记录, 0.02 MB)
✅  已加载 10_built_environment: buildings.shp (189600 条记录, 54.22 MB)
✅  已加载 11_consumption_facilities: consumption_data.shp (2889 条记录, 0.08 MB)
✅  已加载 12_consumption_landuse_mix: landuse_mix.shp (7124 条记录, 2.28 MB)
✅  已加载 13_waste_treatment: impianti_smaltimento_rifiuti.shp (10 条记录, 0.00 MB)


# 创建网格

In [3]:
# # 设置格网大小（单位和你的坐标系一致，比如米）
# grid_size = 1000  # 每个正方形边长1000米（1公里）
#
# # 获取工厂数据的包络范围 (bounding box)
# minx, miny, maxx, maxy = gdfs["00_Boundaries"].total_bounds
#
# # 创建格网
# grid_cells = []
# x_left = minx
# while x_left < maxx:
#     y_bottom = miny
#     while y_bottom < maxy:
#         cell = box(x_left, y_bottom, x_left + grid_size, y_bottom + grid_size)
#         grid_cells.append(cell)
#         y_bottom += grid_size
#     x_left += grid_size
#
# # 转成GeoDataFrame
# grid = gpd.GeoDataFrame({'geometry': grid_cells})
grid= gpd.read_file(r"D:\Architecture\AAA-Master\25Spring\ISUF\UrbanFood\origin_data\grid_entropy.geojson")

## 空间连接，存储为 joined_grid.geojson

In [8]:

# ├── 00_Boundaries/                     # 边界
# │
# ├── 01_production_farming/             # 农作物（可耕地）
# │
# ├── 02_production_aquaculture/         # 水产养殖（海岸泻湖 + 河口）
# │
# ├── 03_production_livestock/           # 畜牧养殖（草场）
# │
# ├── 04_transformation_industry/        # 食品加工厂（GOOGLE 来源）
# │
# ├── 05_transformation_energy/          # 水厂、电厂、供热等能源基础设施
# │
# ├── 06_logistics_transportation/        # 公共交通 + 港口 + 仓储 + 配送中心
# │
# ├── 07_logistics_roads/                # 可通行道路（MultiDiGraph）
# │
# ├── 08_supply_retail/                  # 超市、市场、批发商等零售终端
# │
# ├── 09_supply_public_services/         # 公共服务设施：学校、医疗、福利
# │
# ├── 10_built_environment/              # 建成区信息（居住、商业、工业）
# │
# ├── 11_consumption_facilities/         # 餐饮、酒店、住宿类消费终端
# │
# ├── 12_consumption_landuse_mix/        # 土地混合度计算所需的 landuse 数据
# │
# ├── 13_waste_treatment/                # 废弃物处理设施：垃圾厂、废水厂
# 计算面
grid = tool.area_polygons_in_grid(gdfs["01_production_farming"], grid, "farming_area")
grid = tool.area_polygons_in_grid(gdfs["02_production_aquaculture"], grid, "farming_area")
grid = tool.area_polygons_in_grid(gdfs["03_production_livestock"], grid, "pasture_area")
grid = tool.area_polygons_in_grid(gdfs["05_transformation_energy"], grid, "energy_area")
grid = tool.area_polygons_in_grid(gdfs["06_logistics_transportation"], grid, "transportation_area")
grid = tool.area_polygons_in_grid(gdfs["10_built_environment"], grid, "built_area")
grid = tool.area_polygons_in_grid(gdfs["12_consumption_landuse_mix"], grid, "landuse_area")
# 计算各个landuse
gdf_landuse=gdfs["12_consumption_landuse_mix"]
landuse_types = gdf_landuse["landuse"].unique()
for landuse_type in landuse_types:
    sub_gdf = gdf_landuse[gdf_landuse["landuse"] == landuse_type]
    column_name = f"{landuse_type}_area"
    grid = tool.area_polygons_in_grid(sub_gdf, grid, column_name)
# 计算点
grid = tool.count_points_in_grid(gdfs["04_transformation_industry"], grid, "factory_count")
grid = tool.count_points_in_grid(gdfs["08_supply_retail"], grid, "retail_count")
grid = tool.count_points_in_grid(gdfs["09_supply_public_services"], grid, "public_services")
grid = tool.count_points_in_grid(gdfs["11_consumption_facilities"], grid, "consumption_count")
grid = tool.count_points_in_grid(gdfs["13_waste_treatment"], grid, "waste_treatment")
# 计算线
grid = tool.length_lines_in_grid(gdfs["07_logistics_roads"], grid, "road_length")
# 道路交叉点
gdf_intersections = tool.extract_road_intersections(gdfs["07_logistics_roads"])
grid = tool.count_points_in_grid(gdf_intersections, grid, "intersection_count")
# 其他
grid.to_file("origin_data/joined_grid.geojson")

  Calculating area for farming_area...


100%|██████████| 3576/3576 [00:00<00:00, 5516.53it/s]


  Calculating area for farming_area...



KeyboardInterrupt



In [8]:
variables = ["farming_area","farming_area","pasture_area","building_area","factory_count","public_services","waste_treatment","road_length"]

fig, axs = plt.subplots(2, 4, figsize=(18, 6))
axs = axs.flatten()
for i, var in enumerate(variables):
    grid.plot(column=var, ax=axs[i], cmap='OrRd', legend=True, edgecolor='grey', linewidth=0.2)
    axs[i].set_title(f'{var} per Grid', fontsize=12)
    axs[i].axis('off')

plt.tight_layout()
plt.show()

     grid_id  entropy                                           geometry  \
0          0      0.0  POLYGON ((738456.093 4999587.787, 738456.093 5...   
1          1      0.0  POLYGON ((738456.093 5000587.787, 738456.093 5...   
2          2      0.0  POLYGON ((738456.093 5001587.787, 738456.093 5...   
3          3      0.0  POLYGON ((738456.093 5002587.787, 738456.093 5...   
4          4      0.0  POLYGON ((738456.093 5003587.787, 738456.093 5...   
...      ...      ...                                                ...   
3871    3871      0.0  POLYGON ((794456.093 5062587.787, 794456.093 5...   
3872    3872      0.0  POLYGON ((794456.093 5063587.787, 794456.093 5...   
3873    3873      0.0  POLYGON ((794456.093 5064587.787, 794456.093 5...   
3874    3874      0.0  POLYGON ((794456.093 5065587.787, 794456.093 5...   
3875    3875      0.0  POLYGON ((794456.093 5066587.787, 794456.093 5...   

      farming_area  pasture_area  energy_area  transportation_area  \
0              0.

AttributeError: 'GeoSeries' object has no attribute 'iteritems'

In [ ]:
import seaborn as sns
import pandas as pd

# 只提取需要分析的字段
df_analysis = grid[['factory_count', 'public_services','waste_treatment','road_length', 'industrial_area','agriculture_area','water_area']]
print(df_analysis)
# 计算相关性矩阵
corr_matrix = df_analysis.corr()
corr_matrix.to_csv("correlation_matrix.csv")
# 热力图可视化
plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_analysis)
kmeans = KMeans(n_clusters=6, random_state=0)
labels = kmeans.fit_predict(X_scaled)
grid['cluster'] = labels

fig, ax = plt.subplots(figsize=(10, 10))
grid.plot(column='cluster', cmap='Set1', legend=True, ax=ax, edgecolor='grey')
ax.set_title("K-Means Clustering by Urban Structure")
ax.axis('off')
plt.tight_layout()
plt.show()

cluster_means = pd.DataFrame(X_scaled, columns=df_analysis.columns)
cluster_means['cluster'] = labels
result = cluster_means.groupby('cluster').mean()
print(result)